In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
; Recognizer: is this a natural number in our encoding?
(defun natp* (n)
  (if (equal n 0)
      t
      (and (consp n)
           (equal (car n) 'S)
           (natp* (cdr n)))))


The admission of NATP* is trivial, using the relation O< (which is
known to be well-founded on the domain recognized by O-P) and the measure
(ACL2-COUNT N).  We observe that the type of NATP* is described by
the theorem (OR (EQUAL (NATP* N) T) (EQUAL (NATP* N) NIL)).  

Summary
Form:  ( DEFUN NATP* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 NATP*


In [3]:
; Constructor: O (zero)
(defun O ()
  0)


Since O is non-recursive, its admission is trivial.  We observe that
the type of O is described by the theorem (EQUAL (O) 0).  

Summary
Form:  ( DEFUN O ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 O


In [4]:
; Constructor: S (successor)
(defun S (n)
  (declare (xargs :guard (natp* n)
                  :verify-guards nil))
  (cons 'S n))


Since S is non-recursive, its admission is trivial.  We observe that
the type of S is described by the theorem (CONSP (S N)).  We used primitive
type reasoning.

Summary
Form:  ( DEFUN S ...)
Rules: ((:FAKE-RUNE-FOR-TYPE-SET NIL))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 S


In [5]:
; Addition: defined recursively
; In Coq: Fixpoint plus (n : nat) (m : nat) : nat :=
;   match n with
;   | O => m
;   | S n' => S (plus n' m)
;   end.
(defun plus* (n m)
  (declare (xargs :guard (and (natp* n) (natp* m))
                  :verify-guards nil))
  (if (or (not (natp* n)) (equal n 0))
      m
      (S (plus* (cdr n) m))))


For the admission of PLUS* we will use the relation O< (which is known
to be well-founded on the domain recognized by O-P) and the measure
(ACL2-COUNT N).  The non-trivial part of the measure conjecture is

Goal
(IMPLIES (NOT (OR (NOT (NATP* N)) (EQUAL N 0)))
         (O< (ACL2-COUNT (CDR N))
             (ACL2-COUNT N))).
Goal'
Goal''

Q.E.D.

That completes the proof of the measure theorem for PLUS*.  Thus, we
admit this function under the principle of definition.  We observe
that the type of PLUS* is described by the theorem 
(OR (CONSP (PLUS* N M)) (EQUAL (PLUS* N M) M)).  We used the :type-
prescription rule S.

Summary
Form:  ( DEFUN PLUS* ...)
Rules: ((:DEFINITION ACL2-COUNT)
        (:DEFINITION FIX)
        (:DEFINITION NATP*)
        (:DEFINITION NOT)
        (:DEFINITION O-FINP)
        (:DEFINITION O<)
        (:EXECUTABLE-COUNTERPART ACL2-COUNT)
        (:EXECUTABLE-COUNTERPART NOT)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
       

In [6]:
; Multiplication: defined recursively
; In Coq: Fixpoint mult (n m : nat) : nat :=
;   match n with
;   | O => O
;   | S n' => plus m (mult n' m)
;   end.
(defun mult* (n m)
  (declare (xargs :guard (and (natp* n) (natp* m))
                  :verify-guards nil))
  (if (or (not (natp* n)) (equal n 0))
      0
      (plus* m (mult* (cdr n) m))))


For the admission of MULT* we will use the relation O< (which is known
to be well-founded on the domain recognized by O-P) and the measure
(ACL2-COUNT N).  The non-trivial part of the measure conjecture is

Goal
(IMPLIES (NOT (OR (NOT (NATP* N)) (EQUAL N 0)))
         (O< (ACL2-COUNT (CDR N))
             (ACL2-COUNT N))).
Goal'
Goal''

Q.E.D.

That completes the proof of the measure theorem for MULT*.  Thus, we
admit this function under the principle of definition.  We observe
that the type of MULT* is described by the theorem 
(OR (EQUAL (MULT* N M) 0) (CONSP (MULT* N M))).  We used the :type-
prescription rule PLUS*.

Summary
Form:  ( DEFUN MULT* ...)
Rules: ((:DEFINITION ACL2-COUNT)
        (:DEFINITION FIX)
        (:DEFINITION NATP*)
        (:DEFINITION NOT)
        (:DEFINITION O-FINP)
        (:DEFINITION O<)
        (:EXECUTABLE-COUNTERPART ACL2-COUNT)
        (:EXECUTABLE-COUNTERPART NOT)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
   

In [7]:
; Convert built-in natural to our representation
(defun nat-to-nat* (n)
  (declare (xargs :guard (natp n)
                  :verify-guards nil))
  (if (zp n)
      0
      (S (nat-to-nat* (- n 1)))))


The admission of NAT-TO-NAT* is trivial, using the relation O< (which
is known to be well-founded on the domain recognized by O-P) and the
measure (ACL2-COUNT N).  We observe that the type of NAT-TO-NAT* is
described by the theorem 
(OR (EQUAL (NAT-TO-NAT* N) 0) (CONSP (NAT-TO-NAT* N))).  We used the
:type-prescription rule S.

Summary
Form:  ( DEFUN NAT-TO-NAT* ...)
Rules: ((:TYPE-PRESCRIPTION S))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 NAT-TO-NAT*


In [8]:
; Convert our representation to built-in natural
(defun nat*-to-nat (n)
  (declare (xargs :guard (natp* n)
                  :verify-guards nil))
  (if (or (not (natp* n)) (equal n 0))
      0
      (+ 1 (nat*-to-nat (cdr n)))))


For the admission of NAT*-TO-NAT we will use the relation O< (which
is known to be well-founded on the domain recognized by O-P) and the
measure (ACL2-COUNT N).  The non-trivial part of the measure conjecture
is

Goal
(IMPLIES (NOT (OR (NOT (NATP* N)) (EQUAL N 0)))
         (O< (ACL2-COUNT (CDR N))
             (ACL2-COUNT N))).
Goal'
Goal''

Q.E.D.

That completes the proof of the measure theorem for NAT*-TO-NAT.  Thus,
we admit this function under the principle of definition.  We observe
that the type of NAT*-TO-NAT is described by the theorem 
(AND (INTEGERP (NAT*-TO-NAT N)) (<= 0 (NAT*-TO-NAT N))).  We used primitive
type reasoning.

Summary
Form:  ( DEFUN NAT*-TO-NAT ...)
Rules: ((:DEFINITION ACL2-COUNT)
        (:DEFINITION FIX)
        (:DEFINITION NATP*)
        (:DEFINITION NOT)
        (:DEFINITION O-FINP)
        (:DEFINITION O<)
        (:EXECUTABLE-COUNTERPART ACL2-COUNT)
        (:EXECUTABLE-COUNTERPART NOT)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM)
        (:FAKE-RUN

In [9]:
; Theorem plus_O_n: ∀ n, 0 + n = n
; Source: Software Foundations Basics.v, plus_O_n
(defthm plus*-O-n
  (implies (natp* n)
           (equal (plus* 0 n) n)))


Q.E.D.

Summary
Form:  ( DEFTHM PLUS*-O-N ...)
Rules: ((:DEFINITION PLUS*)
        (:EXECUTABLE-COUNTERPART EQUAL)
        (:EXECUTABLE-COUNTERPART NATP*)
        (:FAKE-RUNE-FOR-TYPE-SET NIL))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  60
 PLUS*-O-N


In [10]:
; Theorem plus_1_l: ∀ n, 1 + n = S n
; Source: Software Foundations Basics.v, plus_1_l
(defthm plus*-1-l
  (implies (natp* n)
           (equal (plus* (S 0) n)
                  (S n))))


ACL2 Warning [Non-rec] in ( DEFTHM PLUS*-1-L ...):  A :REWRITE rule
generated from PLUS*-1-L will be triggered only by terms containing
the function symbol S, which has a non-recursive definition.  Unless
this definition is disabled, this rule is unlikely ever to be used.

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM PLUS*-1-L ...)
Rules: ((:DEFINITION PLUS*)
        (:DEFINITION S)
        (:EXECUTABLE-COUNTERPART CDR)
        (:EXECUTABLE-COUNTERPART EQUAL)
        (:EXECUTABLE-COUNTERPART NATP*)
        (:EXECUTABLE-COUNTERPART S)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE PLUS*-O-N)
        (:TYPE-PRESCRIPTION NATP*))
Warnings:  Non-rec
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  77
 PLUS*-1-L


In [11]:
; Theorem mult_0_l: ∀ n, 0 * n = 0
; Source: Software Foundations Basics.v, mult_0_l
(defthm mult*-0-l
  (implies (natp* n)
           (equal (mult* 0 n) 0)))


Q.E.D.

Summary
Form:  ( DEFTHM MULT*-0-L ...)
Rules: ((:DEFINITION MULT*)
        (:EXECUTABLE-COUNTERPART EQUAL)
        (:EXECUTABLE-COUNTERPART NATP*))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  60
 MULT*-0-L


In [12]:
; Theorem plus_id_exercise: ∀ n m, n = m → n + n = m + m
; Source: Software Foundations Basics.v, plus_id_exercise
(defthm plus*-id-exercise
  (implies (and (natp* n)
                (natp* m)
                (equal n m))
           (equal (plus* n n)
                  (plus* m m))))


ACL2 Warning [Free] in ( DEFTHM PLUS*-ID-EXERCISE ...):  A :REWRITE
rule generated from PLUS*-ID-EXERCISE contains the free variable M.
This variable will be chosen by searching for an instance of (NATP* M)
in the context of the term being rewritten.  This is generally a severe
restriction on the applicability of a :REWRITE rule.  See :DOC free-
variables.


Q.E.D.

Summary
Form:  ( DEFTHM PLUS*-ID-EXERCISE ...)
Rules: ((:FAKE-RUNE-FOR-TYPE-SET NIL))
Warnings:  Free
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  62
 PLUS*-ID-EXERCISE


In [13]:
; Theorem plus_n_O: ∀ n, n + 0 = n
; Source: Software Foundations Induction.v, plus_n_O
(defthm plus*-n-O
  (implies (natp* n)
           (equal (plus* n 0) n)))


*1 (the initial Goal, a key checkpoint) is pushed for proof by induction.

Perhaps we can prove *1 by induction.  Two induction schemes are suggested
by this conjecture.  These merge into one derived induction scheme.

We will induct according to a scheme suggested by (PLUS* N 0), while
accommodating (NATP* N).

These suggestions were produced using the :induction rules NATP* and
PLUS*.  If we let (:P N) denote *1 above then the induction scheme
we'll use is
(AND (IMPLIES (AND (NOT (OR (NOT (NATP* N)) (EQUAL N 0)))
                   (:P (CDR N)))
              (:P N))
     (IMPLIES (OR (NOT (NATP* N)) (EQUAL N 0))
              (:P N))).
This induction is justified by the same argument used to admit PLUS*.
When applied to the goal at hand the above induction scheme produces
three nontautological subgoals.
Subgoal *1/3
Subgoal *1/2
Subgoal *1/1

*1 is COMPLETED!
Thus key checkpoint Goal is COMPLETED!

Q.E.D.

Summary
Form:  ( DEFTHM PLUS*-N-O ...)
Rules: ((:DEFINITION NATP*)
        (

In [14]:
; Theorem mult_0_r: ∀ n, n * 0 = 0
; Source: Software Foundations Induction.v, mult_0_r
(defthm mult*-0-r
  (implies (natp* n)
           (equal (mult* n 0) 0)))


*1 (the initial Goal, a key checkpoint) is pushed for proof by induction.

Perhaps we can prove *1 by induction.  Two induction schemes are suggested
by this conjecture.  These merge into one derived induction scheme.

We will induct according to a scheme suggested by (MULT* N 0), while
accommodating (NATP* N).

These suggestions were produced using the :induction rules MULT* and
NATP*.  If we let (:P N) denote *1 above then the induction scheme
we'll use is
(AND (IMPLIES (AND (NOT (OR (NOT (NATP* N)) (EQUAL N 0)))
                   (:P (CDR N)))
              (:P N))
     (IMPLIES (OR (NOT (NATP* N)) (EQUAL N 0))
              (:P N))).
This induction is justified by the same argument used to admit MULT*.
When applied to the goal at hand the above induction scheme produces
three nontautological subgoals.
Subgoal *1/3
Subgoal *1/2
Subgoal *1/1

*1 is COMPLETED!
Thus key checkpoint Goal is COMPLETED!

Q.E.D.

Summary
Form:  ( DEFTHM MULT*-0-R ...)
Rules: ((:DEFINITION MULT*)
        (

In [15]:
; Theorem plus_n_Sm: ∀ n m, S(n + m) = n + S(m)
; Source: Software Foundations Induction.v, plus_n_Sm
(defthm plus*-n-Sm
  (implies (and (natp* n)
                (natp* m))
           (equal (S (plus* n m))
                  (plus* n (S m)))))


ACL2 Warning [Non-rec] in ( DEFTHM PLUS*-N-SM ...):  A :REWRITE rule
generated from PLUS*-N-SM will be triggered only by terms containing
the function symbol S, which has a non-recursive definition.  Unless
this definition is disabled, this rule is unlikely ever to be used.


ACL2 Warning [Subsume] in ( DEFTHM PLUS*-N-SM ...):  The previously
added rule S subsumes a newly proposed :REWRITE rule generated from
PLUS*-N-SM, in the sense that the old rule rewrites a more general
target.  Because the new rule will be tried first, it may nonetheless
find application.

Goal'

([ A key checkpoint:

Goal'
(IMPLIES (AND (NATP* N) (NATP* M))
         (EQUAL (CONS 'S (PLUS* N M))
                (PLUS* N (CONS 'S M))))

*1 (Goal') is pushed for proof by induction.

])

Perhaps we can prove *1 by induction.  Four induction schemes are suggested
by this conjecture.  Subsumption reduces that number to three.  These
merge into two derived induction schemes.  However, one of these is
flawed and so we 

In [16]:
; Theorem plus_comm: ∀ n m, n + m = m + n
; Source: Software Foundations Induction.v, plus_comm
(defthm plus*-comm
  (implies (and (natp* n)
                (natp* m))
           (equal (plus* n m)
                  (plus* m n))))


*1 (the initial Goal, a key checkpoint) is pushed for proof by induction.

Perhaps we can prove *1 by induction.  Four induction schemes are suggested
by this conjecture.  These merge into two derived induction schemes.
We will choose arbitrarily among these.  

We will induct according to a scheme suggested by (PLUS* N M), while
accommodating (NATP* N).

These suggestions were produced using the :induction rules NATP* and
PLUS*.  If we let (:P M N) denote *1 above then the induction scheme
we'll use is
(AND (IMPLIES (AND (NOT (OR (NOT (NATP* N)) (EQUAL N 0)))
                   (:P M (CDR N)))
              (:P M N))
     (IMPLIES (OR (NOT (NATP* N)) (EQUAL N 0))
              (:P M N))).
This induction is justified by the same argument used to admit PLUS*.
When applied to the goal at hand the above induction scheme produces
three nontautological subgoals.
Subgoal *1/3
Subgoal *1/3'
Subgoal *1/3''
Subgoal *1/3'''
Subgoal *1/3'4'
Subgoal *1/2
Subgoal *1/1

*1 is COMPLETED!
Thus key ch

In [17]:
; Helper: natp* implies the conversion produces natp
(defthm nat*-to-nat-is-natp
  (implies (natp* n)
           (natp (nat*-to-nat n))))


ACL2 Warning [Non-rec] in ( DEFTHM NAT*-TO-NAT-IS-NATP ...):  A :REWRITE
rule generated from NAT*-TO-NAT-IS-NATP will be triggered only by terms
containing the function symbol NATP, which has a non-recursive definition.
Unless this definition is disabled, this rule is unlikely ever to be
used.


Q.E.D.

The storage of NAT*-TO-NAT-IS-NATP depends upon the :compound-recognizer
rule NATP-COMPOUND-RECOGNIZER and the :type-prescription rule NAT*-TO-NAT.

Summary
Form:  ( DEFTHM NAT*-TO-NAT-IS-NATP ...)
Rules: ((:COMPOUND-RECOGNIZER NATP-COMPOUND-RECOGNIZER)
        (:TYPE-PRESCRIPTION NAT*-TO-NAT))
Warnings:  Non-rec
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 NAT*-TO-NAT-IS-NATP


In [18]:
; Helper: S produces valid natp*
(local
 (defthm s-preserves-natp*
   (implies (natp* n)
            (natp* (S n)))))


ACL2 Warning [Non-rec] in ( DEFTHM S-PRESERVES-NATP* ...):  A :REWRITE
rule generated from S-PRESERVES-NATP* will be triggered only by terms
containing the function symbol S, which has a non-recursive definition.
Unless this definition is disabled, this rule is unlikely ever to be
used.

Goal'

Q.E.D.

The storage of S-PRESERVES-NATP* depends upon the :type-prescription
rule NATP*.

Summary
Form:  ( DEFTHM S-PRESERVES-NATP* ...)
Rules: ((:DEFINITION NATP*)
        (:DEFINITION S)
        (:EXECUTABLE-COUNTERPART EQUAL)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE CAR-CONS)
        (:REWRITE CDR-CONS)
        (:TYPE-PRESCRIPTION NATP*))
Warnings:  Non-rec
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  68
 S-PRESERVES-NATP*


In [19]:
; Helper: S increments the natural number representation
(local
 (defthm nat*-to-nat-of-s
   (implies (natp* n)
            (equal (nat*-to-nat (S n))
                   (+ 1 (nat*-to-nat n))))))


ACL2 Warning [Non-rec] in ( DEFTHM NAT*-TO-NAT-OF-S ...):  A :REWRITE
rule generated from NAT*-TO-NAT-OF-S will be triggered only by terms
containing the function symbol S, which has a non-recursive definition.
Unless this definition is disabled, this rule is unlikely ever to be
used.

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM NAT*-TO-NAT-OF-S ...)
Rules: ((:DEFINITION NAT*-TO-NAT)
        (:DEFINITION NATP*)
        (:DEFINITION S)
        (:EXECUTABLE-COUNTERPART EQUAL)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE CAR-CONS)
        (:REWRITE CDR-CONS)
        (:TYPE-PRESCRIPTION NATP*))
Warnings:  Non-rec
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  166
 NAT*-TO-NAT-OF-S


In [20]:
; Helper: plus* produces valid natp*
(local
 (defthm plus*-preserves-natp*
   (implies (and (natp* n) (natp* m))
            (natp* (plus* n m)))
   :hints (("Goal" :in-theory (disable plus*-comm plus*-n-sm)))))


*1 (the initial Goal, a key checkpoint) is pushed for proof by induction.

Perhaps we can prove *1 by induction.  Three induction schemes are
suggested by this conjecture.  These merge into two derived induction
schemes.  However, one of these is flawed and so we are left with one
viable candidate.  

We will induct according to a scheme suggested by (PLUS* N M), while
accommodating (NATP* N).

These suggestions were produced using the :induction rules NATP* and
PLUS*.  If we let (:P M N) denote *1 above then the induction scheme
we'll use is
(AND (IMPLIES (AND (NOT (OR (NOT (NATP* N)) (EQUAL N 0)))
                   (:P M (CDR N)))
              (:P M N))
     (IMPLIES (OR (NOT (NATP* N)) (EQUAL N 0))
              (:P M N))).
This induction is justified by the same argument used to admit PLUS*.
When applied to the goal at hand the above induction scheme produces
three nontautological subgoals.
Subgoal *1/3
Subgoal *1/2
Subgoal *1/1

*1 is COMPLETED!
Thus key checkpoint Goal is COMP

In [21]:
; Helper: nat-to-nat* produces valid natp*
(defthm nat-to-nat*-is-natp*
  (implies (natp n)
           (natp* (nat-to-nat* n))))

Goal'

([ A key checkpoint:

Goal'
(IMPLIES (AND (INTEGERP N) (<= 0 N))
         (NATP* (NAT-TO-NAT* N)))

*1 (Goal') is pushed for proof by induction.

])

Perhaps we can prove *1 by induction.  One induction scheme is suggested
by this conjecture.  

We will induct according to a scheme suggested by (NAT-TO-NAT* N).

This suggestion was produced using the :induction rule NAT-TO-NAT*.
If we let (:P N) denote *1 above then the induction scheme we'll use
is
(AND (IMPLIES (AND (NOT (ZP N)) (:P (+ -1 N)))
              (:P N))
     (IMPLIES (ZP N) (:P N))).
This induction is justified by the same argument used to admit NAT-TO-NAT*.
When applied to the goal at hand the above induction scheme produces
four nontautological subgoals.
Subgoal *1/4
Subgoal *1/3
Subgoal *1/2
Subgoal *1/1

*1 is COMPLETED!
Thus key checkpoint Goal' is COMPLETED!

Q.E.D.

The storage of NAT-TO-NAT*-IS-NATP* depends upon the :type-prescription
rule NATP*.

Summary
Form:  ( DEFTHM NAT-TO-NAT*-IS-NATP* ...)
Rules: ((

In [22]:
; Helper: conversion round-trip preserves values
(defthm nat*-to-nat-of-nat-to-nat*
  (implies (natp n)
           (equal (nat*-to-nat (nat-to-nat* n))
                  n)))

Goal'

([ A key checkpoint:

Goal'
(IMPLIES (AND (INTEGERP N) (<= 0 N))
         (EQUAL (NAT*-TO-NAT (NAT-TO-NAT* N)) N))

*1 (Goal') is pushed for proof by induction.

])

Perhaps we can prove *1 by induction.  One induction scheme is suggested
by this conjecture.  

We will induct according to a scheme suggested by (NAT-TO-NAT* N).

This suggestion was produced using the :induction rule NAT-TO-NAT*.
If we let (:P N) denote *1 above then the induction scheme we'll use
is
(AND (IMPLIES (AND (NOT (ZP N)) (:P (+ -1 N)))
              (:P N))
     (IMPLIES (ZP N) (:P N))).
This induction is justified by the same argument used to admit NAT-TO-NAT*.
When applied to the goal at hand the above induction scheme produces
four nontautological subgoals.
Subgoal *1/4
Subgoal *1/3
Subgoal *1/2
Subgoal *1/1

*1 is COMPLETED!
Thus key checkpoint Goal' is COMPLETED!

Q.E.D.

Summary
Form:  ( DEFTHM NAT*-TO-NAT-OF-NAT-TO-NAT* ...)
Rules: ((:COMPOUND-RECOGNIZER NATP-COMPOUND-RECOGNIZER)
        (:COMPOU